# Data Analysis Workshop

## Tutorial II: Add/Remove dataset to/from Freva Databrowser

<div style="
  border-left: 6px solid rgb(236, 114, 0);
  background-color:rgb(253, 231, 157);
  color:rgb(19, 19, 18);
  padding: 1em;
  font-size: 110%;
  border-radius: 4px;
  margin: 1em 0;
">
⚠️ <strong>ATTENTION</strong>: if the kernel is not already set, please change it to the <code>DA Workshop (shell)</code>
</div>

<div style="border-left: 4px solid #0366d6; padding: 0.5em; background-color: #deecfc;">
  ℹ️ If you want to know more about these topics please refer to:
  <ul>
  <li> <a href='https://freva-org.github.io/freva-nextgen/auth/index.html#' target="_blank">authentication</a></li>
  <li>adding and deleting user data <a href="https://freva-org.github.io/freva-nextgen/databrowser/cli.html#adding-and-deleting-user-data">via <code>databrowser user-data</code> </a></li>
</ul>  
</div>

<br>
<br>

So far in this tutorial, we've learned how to search for datasets, inspect metadata, and access data directly from Freva's indexed resources. But what if you want to work with data that isn't yet indexed, like your own model simulations or derived statistics?

In this next section, we'll walk through how to register your own dataset with Freva's databrowser:

1. Prepare your data
   We'll use a sample file of extreme‐value statistics (dummy sea surface temperature extremes), but you can point to any local or remote dataset you've generated.

2. Open the dataset  
   Loading the file locally allows Freva to auto-extract core metadata attributes (e.g., dimensions, coordinate variables).

3. Define missing metadata
   For any required fields not present in the file (project, experiment, realm, etc.), we'll set the appropriate Freva attributes.

4. Ingest into Freva
   You'll see how to add the dataset to the databrowser index—and afterward, how to remove it again, to demonstrate both ingestion and deletion workflows.

By the end of this section, you'll be able to make your own simulations and analyses discoverable alongside Freva's curated archives.

Let's start creating a dummy surface sea temperature monthly file for the tropical Pacific region:

In [ ]:
USER=$(whoami)
OUTFILE="dummy_sst_${USER}.nc"
GRIDFILE="$(mktemp)"
cat > "$GRIDFILE" << EOF
gridtype = lonlat
xsize    = 171
ysize    = 121
xfirst   = 120
xinc     = 1
yfirst   = -30
yinc     = 0.5
xname    = lon
yname    = lat
xunits   = degrees_east
yunits   = degrees_north
EOF
cdo -f nc const,0,"$GRIDFILE" base.nc
cdo -f nc \
    -setcalendar,standard \
    -settaxis,2025-01-01,00:00:00,1month \
    -expr,'sst=28-0.006*(clon(const)-230)' \
    -setattribute,sst@long_name="Idealized Pacific SST" \
    -setattribute,sst@units="°C" \
    base.nc "$OUTFILE"
rm -f base.nc "$GRIDFILE"
echo "Wrote $OUTFILE → exists? $( [[ -f $OUTFILE ]] && echo yes || echo no )"

Now we are going to add this dummy SST data to the Freva databrowser

For this purpose we need to first authenticate to be able to be know to the server to be able to add and remove data.
To do so, please follow the folowing instruction to do it:

1. Please head over to the https://www.gems.dkrz.de and then loging to the website.

    <img src="https://github.com/freva-org/Talks/raw/main/talks/DataSearchWorkshop2025/media/token1.png" width=600px>
2. When you are logged in, please click on your username button on top right and choose the `Token Management`.

    <img src="https://github.com/freva-org/Talks/raw/main/talks/DataSearchWorkshop2025/media/token2.png" width=600px>
3. from the buttons on the token management, please click on the `Copy`

    <img src="https://github.com/freva-org/Talks/raw/main/talks/DataSearchWorkshop2025/media/token3.png" width=600px>
4. Paste the copied token in the cell below here `"""TOKEN_SHALL_BE_PASTED_HERE"""`


In [ ]:
# <copy-paste> the token content from the web site below
cat > ~/.freva-access-token.json << 'EOF'

EOF

We will use this token to authenticate ourselves with the `--token-file` parameter.

<br>

Now lets add our newly created dummy data to Freva with the `user-data` function.

Freva is able to infer, from the filename and/or file metadata some of the attributes of this file, such as, e.g. `time_frequency` and `variable`. But I cannot make up for all of the attributes, however, we can explicitly define them before we want to index our file into the databrowser so they are not all empty. We will do it explicitly setting the desired key value pairs manually:

In [ ]:
freva-client databrowser user-data add --path dummy_sst_$USER.nc --facet project="${USER}data" --facet product=stats --facet model=IFS --facet experiment=ETCCDI --facet realm=atmos --token-file ~/.freva-access-token.json

This response shows that the data has been successfully added to Freva. Now we can query the data to see if the data is on Freva

In [ ]:
freva-client databrowser metadata-search --flavour user 

Since we are sure that the data has been added to the Freva, we can remove it via the following command:

In [ ]:
freva-client databrowser user-data delete --search-key project="${USER}data" --search-key product=stats --search-key model=IFS --search-key experiment=ETCCDI --search-key realm=atmos --token-file ~/.freva-access-token.json

For demonstration purposes we can check again:

In [ ]:
freva-client databrowser metadata-search --flavour user 